In [13]:
#imports
import json
from pathlib import Path
from tqdm import tqdm 
import citation_parser


In [ ]:
# generates a txt file with the first N entries of the shard



path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\shard1.jsonl")  # adjust as needed
output_path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\output_text.txt")


N = 20
entries = []

with path.open("r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= N:
            break
        obj = json.loads(line)
        entries.append(obj)

with output_path.open("w", encoding="utf-8") as out:
    for i, obj in enumerate(entries):
        out.write(f"\n===== ENTRY {i} =====")
        out.write(json.dumps(obj, indent=2, ensure_ascii=False))  # truncate long ones
        out.write("\n")

In [12]:
#saves the parsed citations from the first N entries of the shard
path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\shard1.jsonl")
output_path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\citations_shard1_complete.json")
N = 60550
with output_path.open("w", encoding="utf-8") as out:
    for paper in citation_parser.iterate_papers(path, N):
        citations = citation_parser.get_citations(paper)
        out.write(json.dumps(citations, ensure_ascii=False, indent=2))
        out.write("\n\n")

Exception ignored in: <function tqdm.__del__ at 0x000002B27AA3BBA0>
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\std.py", line 1148, in __del__
    self.close()
  File "c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\notebook.py", line 279, in close
    self.disp(bar_style='danger', check_delay=False)
    ^^^^^^^^^
AttributeError: 'tqdm_notebook' object has no attribute 'disp'


AttributeError: 'NoneType' object has no attribute 'get'

In [14]:
from pathlib import Path

input_path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\shard1.jsonl")
output_path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\citations_shard1_complete.json")

# Count papers in the input shard
with input_path.open("r", encoding="utf-8") as f:
    input_papers = sum(1 for line in f if line.strip())
print("Input papers:", input_papers)

# Count papers in the output (you have one JSON object per paper,
# and blank lines between, so count non-empty lines)
with output_path.open("r", encoding="utf-8") as f:
    output_papers = sum(1 for line in f if line.strip())
print("Output papers:", output_papers)


Input papers: 60550
Output papers: 49357120


In [15]:
from pathlib import Path

output_path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\citations_shard1_complete.json")

with output_path.open("r", encoding="utf-8") as f:
    num_papers_out = sum(1 for line in f if not line.strip())

print("Papers in output:", num_papers_out)


Papers in output: 59511


In [17]:
from pathlib import Path
import json

shard_path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\shard1.jsonl")

shard_ids = set()
with shard_path.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        paper = json.loads(line)
        shard_ids.add(paper["corpusid"])

print("Number of papers in shard:", len(shard_ids))


Number of papers in shard: 60550


In [22]:
from pathlib import Path
import json

citations_path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\citations_shard1.json")
filtered_path  = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\citations_shard1_intra_shard.jsonl")  # new JSONL output

def iter_parsed_citation_papers(path):
    """Yield one parsed-citation paper (dict) at a time from citations_shard1.json."""
    buf = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            # blank line = separator between JSON objects
            if not line.strip():
                if buf:
                    yield json.loads("".join(buf))
                    buf = []
            else:
                buf.append(line)
        # last object (if file doesn't end with blank line)
        if buf:
            yield json.loads("".join(buf))

with filtered_path.open("w", encoding="utf-8") as out:
    for paper in iter_parsed_citation_papers(citations_path):
        paper_id = paper.get("paper_id")

        # (Probably redundant, but ensures we only keep papers actually in this shard)
        if paper_id not in shard_ids:
            continue

        citations = paper.get("citations", [])
        # keep only citations whose matched_paper_id exists and is also in the shard
        filtered_cits = [
            c for c in citations
            if c.get("matched_paper_id") is not None
            and c["matched_paper_id"] in shard_ids
        ]

        if not filtered_cits:
            # no intra-shard citations → drop this paper
            continue

        paper["citations"] = filtered_cits

        # write 1 JSON object per line (much nicer format)
        out.write(json.dumps(paper, ensure_ascii=False))
        out.write("\n")


In [23]:
from pathlib import Path
import json

shard_path     = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\shard1.jsonl")
citations_path = Path(r"C:\Users\User\Desktop\Julian\Uni\WS 25\AIR\herBERT\Datasets\Training Data\s2orc\shard1\citations_shard1.json")  # or your big file

# 1) collect all corpusid values from the shard
shard_ids = set()
with shard_path.open("r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        paper = json.loads(line)
        cid = paper.get("corpusid")
        if cid is not None:
            shard_ids.add(cid)

print("Shard papers:", len(shard_ids))


# 2) iterate over your parsed citation objects
def iter_parsed_citation_papers(path):
    buf = []
    with path.open("r", encoding="utf-8") as f:
        for line in f:
            if not line.strip():
                if buf:
                    yield json.loads("".join(buf))
                    buf = []
            else:
                buf.append(line)
        if buf:
            yield json.loads("".join(buf))


matched_ids = set()
intra_shard_count = 0

for paper in iter_parsed_citation_papers(citations_path):
    for c in paper.get("citations", []):
        mid = c.get("matched_paper_id")
        if mid is not None:
            matched_ids.add(mid)
            if mid in shard_ids:
                intra_shard_count += 1

print("Distinct matched_paper_id:", len(matched_ids))
print("Number of citations where matched_paper_id is in this shard:", intra_shard_count)


Shard papers: 60550
Distinct matched_paper_id: 477
Number of citations where matched_paper_id is in this shard: 0
